<a href="https://colab.research.google.com/github/tztechno/cc_archive/blob/main/Gaussian_Splat_w_DINO%2BALIKED_OK_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR



# **Fountain: Gaussian Splat w/ DINO+ALIKED**

In [2]:
!pip install kornia
!pip install pycolmap
!pip install lightglue
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 118.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 85.4 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement lightglue (from versions: none)
ERROR: No matching distribution found for lightglue


In [3]:
import subprocess
import sys
import os

INSTALL_DIR = "/content/packages"
os.makedirs(INSTALL_DIR, exist_ok=True)

print("="*60)
print("Building LightGlue only...")
print("="*60)

# Install LightGlue only
# Dependencies (including pycolmap) will not be installed
print("\nInstalling LightGlue (no dependencies)...")
subprocess.run([
    sys.executable, '-m', 'pip', 'install',
    '--target', INSTALL_DIR,
    '--no-deps',  # No dependencies
    'git+https://github.com/cvg/LightGlue.git'
], check=True)

print("\n" + "="*60)
print("✓ Build Complete!")
print("="*60)
print(f"\nPackages built to: {INSTALL_DIR}")
print("\nNote: pycolmap will be installed on-the-fly in the main notebook.")

Building LightGlue only...

Installing LightGlue (no dependencies)...

✓ Build Complete!

Packages built to: /content/packages

Note: pycolmap will be installed on-the-fly in the main notebook.


In [ ]:
def setup_environment():
    """
    Setup environment with clean NumPy installation at the beginning
    """
    print("Setting up environment for Kaggle...")
    WORK_DIR = '/kaggle/working/gaussian_splatting'

    # ========================================================================
    # STEP 0: Clean NumPy installation BEFORE importing anything
    # ========================================================================
    print("="*70)
    print("STEP 0: Fixing NumPy compatibility (clean install)")
    print("="*70)

    try:
        # Uninstall NumPy completely
        print("Uninstalling NumPy 2.x...")
        subprocess.run([
            sys.executable, '-m', 'pip', 'uninstall', '-y', 'numpy'
        ], check=True, capture_output=True)
        print("✓ NumPy uninstalled")

        # Install NumPy 1.x
        print("Installing NumPy 1.x...")
        subprocess.run([
            sys.executable, '-m', 'pip', 'install', 'numpy<2'
        ], check=True, capture_output=True)
        print("✓ NumPy 1.x installed")

        # Reinstall key packages that depend on NumPy
        print("Reinstalling NumPy-dependent packages...")
        packages_to_reinstall = [
            'scikit-learn',
            'scipy',
            'matplotlib',
            'pandas'
        ]

        for pkg in packages_to_reinstall:
            try:
                subprocess.run([
                    sys.executable, '-m', 'pip', 'install', '--force-reinstall',
                    '--no-deps', pkg
                ], check=True, capture_output=True)
                print(f"✓ Reinstalled {pkg}")
            except subprocess.CalledProcessError:
                print(f"⚠ Failed to reinstall {pkg} (may not be critical)")

        # Verify NumPy version
        result = subprocess.run([
            sys.executable, '-c', 'import numpy; print(numpy.__version__)'
        ], capture_output=True, text=True)
        numpy_version = result.stdout.strip()
        print(f"\n✓ NumPy version now: {numpy_version}")

        if numpy_version.startswith('1.'):
            print("✓ NumPy fix successful!")
        else:
            print(f"⚠ Warning: NumPy version is {numpy_version}, expected 1.x")

    except subprocess.CalledProcessError as e:
        print(f"⚠ NumPy fix encountered issues: {e}")
        print("Continuing anyway...")

    # ========================================================================
    # STEP 1: System packages and dependencies
    # ========================================================================
    print("\n" + "="*70)
    print("STEP 1: Installing system packages")
    print("="*70)

    # Virtual display setup
    try:
        print("Setting up virtual display...")
        subprocess.run(['apt-get', 'update', '-qq'], check=True, capture_output=True)
        subprocess.run(['apt-get', 'install', '-y', '-qq', 'xvfb'],
                      check=True, capture_output=True)

        os.environ['QT_QPA_PLATFORM'] = 'offscreen'
        os.environ['DISPLAY'] = ':99'
        subprocess.Popen(['Xvfb', ':99', '-screen', '0', '1024x768x24'],
                         stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print("✓ Virtual display setup")
    except Exception as e:
        print(f"⚠ Virtual display skipped: {e}")

    # Install COLMAP
    print("\nInstalling COLMAP...")
    try:
        subprocess.run(['apt-get', 'install', '-y', '-qq', 'colmap'],
                       check=True, capture_output=True)
        print("✓ COLMAP installed")
    except subprocess.CalledProcessError as e:
        print(f"⚠ COLMAP warning: {e}")

    # Install build dependencies
    print("\nInstalling build dependencies...")
    try:
        subprocess.run([
            'apt-get', 'install', '-y', '-qq',
            'build-essential', 'cmake', 'git', 'libopenblas-dev'
        ], check=True, capture_output=True)
        print("✓ Build dependencies installed")
    except subprocess.CalledProcessError as e:
        print(f"⚠ Build dependencies warning: {e}")

    # ========================================================================
    # STEP 2: Clone Gaussian Splatting repository
    # ========================================================================
    print("\n" + "="*70)
    print("STEP 2: Cloning repository")
    print("="*70)

    if not os.path.exists(WORK_DIR):
        print(f"Cloning to {WORK_DIR}...")
        try:
            subprocess.run([
                'git', 'clone', '--recursive',
                'https://github.com/graphdeco-inria/gaussian-splatting.git',
                WORK_DIR
            ], check=True)
            print("✓ Repository cloned")
        except subprocess.CalledProcessError:
            print("Primary repository failed, trying alternative...")
            try:
                subprocess.run([
                    'git', 'clone', '--recursive',
                    'https://github.com/tztechno/gaussian-splatting.git',
                    WORK_DIR
                ], check=True)
                print("✓ Alternative repository cloned")
            except subprocess.CalledProcessError as e:
                print(f"✗ Both repositories failed: {e}")
                raise
    else:
        print(f"✓ Repository exists at {WORK_DIR}")
        try:
            subprocess.run(['git', 'submodule', 'update', '--init', '--recursive'],
                          cwd=WORK_DIR, check=True, capture_output=True)
            print("✓ Submodules updated")
        except subprocess.CalledProcessError:
            print("⚠ Submodule update failed")

    # ========================================================================
    # STEP 3: Install Python packages
    # ========================================================================
    print("\n" + "="*70)
    print("STEP 3: Installing Python packages")
    print("="*70)

    pip_packages = [
        'torch', 'torchvision', 'torchaudio',
        'plyfile', 'tqdm', 'opencv-python', 'pillow',
        'imageio', 'imageio-ffmpeg', 'tensorboard'
    ]

    for package in pip_packages:
        try:
            subprocess.run([
                sys.executable, '-m', 'pip', 'install', '-q', package
            ], check=True, capture_output=True)
            print(f"✓ Installed {package}")
        except subprocess.CalledProcessError:
            print(f"⚠ Failed to install {package}")

    # ========================================================================
    # STEP 4: Verify CUDA
    # ========================================================================
    print("\n" + "="*70)
    print("STEP 4: Verifying CUDA")
    print("="*70)

    try:
        import torch
        print(f"PyTorch: {torch.__version__}")
        print(f"CUDA available: {torch.cuda.is_available()}")
        if torch.cuda.is_available():
            print(f"CUDA version: {torch.version.cuda}")
            print(f"GPU: {torch.cuda.get_device_name(0)}")
    except ImportError:
        print("⚠ PyTorch not available")

    # ========================================================================
    # STEP 5: Build submodules
    # ========================================================================
    print("\n" + "="*70)
    print("STEP 5: Building submodules")
    print("="*70)

    submodules = [
        ('diff-gaussian-rasterization',
         'https://github.com/graphdeco-inria/diff-gaussian-rasterization.git'),
        ('simple-knn',
         'https://github.com/camenduru/simple-knn.git')
    ]

    for submodule_name, fallback_url in submodules:
        print(f"\n{'-'*70}")
        print(f"Building {submodule_name}...")
        print(f"{'-'*70}")

        submodule_dir = os.path.join(WORK_DIR, 'submodules', submodule_name)

        # Check/clone submodule
        if not os.path.exists(submodule_dir) or not os.listdir(submodule_dir):
            print(f"Cloning {submodule_name}...")
            try:
                subprocess.run(['git', 'clone', fallback_url, submodule_dir],
                              check=True)
                print(f"✓ Cloned {submodule_name}")
            except subprocess.CalledProcessError:
                print(f"✗ Failed to clone {submodule_name}")
                continue

        # Try installation methods in order
        methods = [
            ("pip install", lambda: subprocess.run([
                sys.executable, '-m', 'pip', 'install', submodule_dir
            ], check=True, capture_output=True)),

            ("setup.py install", lambda: subprocess.run([
                sys.executable, 'setup.py', 'install'
            ], cwd=submodule_dir, check=True, capture_output=True)),

            ("git install", lambda: subprocess.run([
                sys.executable, '-m', 'pip', 'install', f'git+{fallback_url}'
            ], check=True, capture_output=True))
        ]

        for method_name, method_func in methods:
            try:
                print(f"Trying {method_name}...")
                method_func()
                print(f"✓ {submodule_name} installed via {method_name}")
                break
            except subprocess.CalledProcessError:
                print(f"✗ {method_name} failed")
        else:
            print(f"⚠ All methods failed for {submodule_name}")

    # ========================================================================
    # STEP 6: Verify installations
    # ========================================================================
    print("\n" + "="*70)
    print("STEP 6: Verifying installations")
    print("="*70)

    all_good = True

    try:
        import diff_gaussian_rasterization
        print("✓ diff_gaussian_rasterization available")
    except ImportError:
        print("✗ diff_gaussian_rasterization NOT FOUND")
        all_good = False

    try:
        import simple_knn
        print("✓ simple_knn available")
    except ImportError:
        print("✗ simple_knn NOT FOUND")
        all_good = False

    # Final summary
    print("\n" + "="*70)
    if all_good:
        print("✓✓✓ SETUP COMPLETE - Ready to run! ✓✓✓")
    else:
        print("⚠⚠⚠ SETUP COMPLETED WITH WARNINGS ⚠⚠⚠")
        print("Some modules may be missing. Training might fail.")
    print(f"Working directory: {WORK_DIR}")
    print("="*70)

    return WORK_DIR


if __name__ == "__main__":
    setup_environment()

Setting up environment for Kaggle...
STEP 0: Fixing NumPy compatibility (clean install)
Uninstalling NumPy 2.x...
✓ NumPy uninstalled
Installing NumPy 1.x...
✓ NumPy 1.x installed
Reinstalling NumPy-dependent packages...
✓ Reinstalled scikit-learn
✓ Reinstalled scipy


In [ ]:

#サイズの異なる画像を扱う
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os
import sys
import subprocess
import shutil
from pathlib import Path
import cv2

# Configuration
# IMAGE_PATH: Path to the image folder
# WORK_DIR: Working directory for Gaussian Splatting repository
# OUTPUT_DIR: Directory for the final video output
# COLMAP_DIR: Directory for COLMAP data

IMAGE_DIR = "/content/drive/MyDrive/your_folder/fountain2"
WORK_DIR = '/content/gaussian_splatting'
OUTPUT_DIR = '/content/output'
COLMAP_DIR = '/content/colmap_data'

ORIGINAL=IMAGE_DIR
RESIZED='/content/resized'

In [ ]:
def normalize_image_sizes(image_dir, output_dir=None, target_size=1200, mode='fit'):
    """
    Resizes all images in a directory while maintaining aspect ratio.

    Args:
        image_dir: Directory containing input images.
        output_dir: Directory to save the processed images. Defaults to image_dir.
        target_size: The desired maximum size for the longer side (or minimum size for the shorter side).
        mode: Resizing mode - 'fit' (fit within target), 'fill' (fill target), or 'pad' (fit with padding).
    """
    if output_dir is None:
        output_dir = image_dir

    os.makedirs(output_dir, exist_ok=True)

    print(f"Normalizing image sizes (mode: {mode}) while maintaining aspect ratio...")

    size_stats = {}
    converted_count = 0

    for img_file in sorted(os.listdir(image_dir)):
        if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        input_path = os.path.join(image_dir, img_file)
        output_path = os.path.join(output_dir, img_file)

        try:
            img = Image.open(input_path)
            original_size = img.size  # (width, height)
            original_aspect = original_size[0] / original_size[1]

            # Record original size for statistics
            size_key = f"{original_size[0]}x{original_size[1]}"
            if size_key not in size_stats:
                size_stats[size_key] = 0
            size_stats[size_key] += 1

            # Resize while maintaining aspect ratio
            if mode == 'fit':
                # Fit within target (長辺をtarget_sizeに合わせてリサイズ)
                if original_size[0] > original_size[1]:  # 横長
                    new_width = target_size
                    new_height = int(target_size / original_aspect)
                else:  # 縦長 or 正方形
                    new_height = target_size
                    new_width = int(target_size * original_aspect)

            elif mode == 'fill':
                # Fill target (短辺をtarget_sizeに合わせてリサイズ)
                if original_size[0] > original_size[1]:  # 横長
                    new_height = target_size
                    new_width = int(target_size * original_aspect)
                else:  # 縦長 or 正方形
                    new_width = target_size
                    new_height = int(target_size / original_aspect)

            elif mode == 'pad':
                # Fit with padding (短辺をtarget_sizeに合わせて、余白を追加)
                if original_size[0] > original_size[1]:  # 横長
                    new_width = target_size
                    new_height = int(target_size / original_aspect)
                else:  # 縦長 or 正方形
                    new_height = target_size
                    new_width = int(target_size * original_aspect)

                # 余白を追加して正方形にする
                img_resized = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
                img_square = Image.new('RGB', (target_size, target_size), (255, 255, 255))
                offset = ((target_size - new_width) // 2, (target_size - new_height) // 2)
                img_square.paste(img_resized, offset)
                img = img_square
                print(f"  ✓ {img_file}: {original_size} → {new_width}x{new_height} (padded to {target_size}x{target_size})")
                img.save(output_path, quality=95)
                converted_count += 1
                continue

            else:
                raise ValueError(f"Unknown mode: {mode}. Use 'fit', 'fill', or 'pad'.")

            # リサイズ実行
            img_resized = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
            img_resized.save(output_path, quality=95)
            converted_count += 1

            print(f"  ✓ {img_file}: {original_size} → {new_width}x{new_height} (aspect ratio: {original_aspect:.2f})")

        except Exception as e:
            print(f"  ✗ Error processing {img_file}: {e}")

    print(f"\nConversion complete: {converted_count} images")
    print(f"Original size distribution: {size_stats}")
    return converted_count



normalize_image_sizes(ORIGINAL, RESIZED, target_size=1000, mode='fit'):

In [ ]:
# =========================================================
# Cell 1: Setup (Revised Version)
# =========================================================
import sys
import os
import subprocess

# Environment settings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("Setting up environment...")
print("="*60)

# =========================================================
# Kaggle defaults (Packages with C-binaries)
# =========================================================
print("\n[1/5] Loading Kaggle default packages...")
import gc
import glob
from pathlib import Path
from tqdm import tqdm

import numpy as np
import h5py
import cv2
import torch
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF

print(f"✓ Defaults loaded: numpy {np.__version__}, torch {torch.__version__}")
print(f"  CUDA Available: {torch.cuda.is_available()}")

# =========================================================
# pycolmap (With C-binaries - Install on-the-fly)
# =========================================================
print("\n[2/5] Installing pycolmap...")
try:
    import pycolmap
    print("✓ pycolmap already available")
except ImportError:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'pycolmap'],
                  check=True)
    import pycolmap
    print("✓ pycolmap installed")

# =========================================================
# LightGlue (Pure Python - Using pre-built package)
# =========================================================
print("\n[3/5] Loading LightGlue...")
PACKAGES_PATH = '/content/lightglue-package-builder/packages'

if os.path.exists(PACKAGES_PATH):
    sys.path.insert(0, PACKAGES_PATH)
    print(f"✓ Using pre-built path: {PACKAGES_PATH}")
else:
    print(f"⚠️  Pre-built not found, installing via pip...")
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q',
                    'git+https://github.com/cvg/LightGlue.git'], check=True)

from lightglue import ALIKED, LightGlue
print("✓ LightGlue loaded")

# =========================================================
# transformers
# =========================================================
print("\n[4/5] Loading transformers...")
try:
    from transformers import AutoImageProcessor, AutoModel
    print("✓ transformers loaded")
except ImportError:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'transformers'],
                  check=True)
    from transformers import AutoImageProcessor, AutoModel
    print("✓ transformers installed")

# =========================================================
# COLMAP (System Package)
# =========================================================
print("\n[5/5] Installing COLMAP system binary...")
subprocess.run(['apt-get', 'update', '-qq'], capture_output=True)
subprocess.run(['apt-get', 'install', '-y', '-qq', 'colmap'], capture_output=True)
print("✓ COLMAP binary installed")

print("\n" + "="*60)
print("✓ All packages ready!")
print("="*60)

In [ ]:
import os
import gc
import glob
import subprocess
from pathlib import Path
from tqdm import tqdm
import numpy as np
import h5py
import sqlite3
import torch
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF
from lightglue import ALIKED, LightGlue
from transformers import AutoImageProcessor, AutoModel
import pycolmap
from PIL import Image


class CONFIG:
    GLOBAL_TOPK = 200
    RATIO_THR = 1.2
    MATCH_THRESH = 10
    N_KEYPOINTS = 2048
    exhaustive_if_less = 20
    min_matches = 15
    max_num_keypoints = 8192
    image_size = 1024
    colmap_camera_model = 'SIMPLE_RADIAL'

In [ ]:
# =========================================================
# COLMAP Database Utilities
# =========================================================
class COLMAPDatabase:
    @staticmethod
    def connect(database_path):
        return COLMAPDatabase(database_path)

    def __init__(self, database_path):
        self.connection = sqlite3.connect(database_path)
        self.cursor = self.connection.cursor()

    def create_tables(self):
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS cameras (
                camera_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                model INTEGER NOT NULL,
                width INTEGER NOT NULL,
                height INTEGER NOT NULL,
                params BLOB,
                prior_focal_length INTEGER NOT NULL
            )
        """)
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS images (
                image_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                name TEXT NOT NULL UNIQUE,
                camera_id INTEGER NOT NULL,
                prior_qw REAL,
                prior_qx REAL,
                prior_qy REAL,
                prior_qz REAL,
                prior_tx REAL,
                prior_ty REAL,
                prior_tz REAL,
                CONSTRAINT image_id_check CHECK(image_id >= 0 and image_id < 2147483647),
                FOREIGN KEY(camera_id) REFERENCES cameras(camera_id)
            )
        """)
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS keypoints (
                image_id INTEGER PRIMARY KEY NOT NULL,
                rows INTEGER NOT NULL,
                cols INTEGER NOT NULL,
                data BLOB,
                FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE
            )
        """)
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS matches (
                pair_id INTEGER PRIMARY KEY NOT NULL,
                rows INTEGER NOT NULL,
                cols INTEGER NOT NULL,
                data BLOB
            )
        """)
        self.cursor.execute("CREATE UNIQUE INDEX IF NOT EXISTS index_name ON images(name)")

    def add_camera(self, model, width, height, params, prior_focal_length=1):
        params_blob = np.array(params, dtype=np.float64).tobytes()
        self.cursor.execute(
            "INSERT INTO cameras VALUES (?, ?, ?, ?, ?, ?)",
            (None, model, width, height, params_blob, prior_focal_length)
        )
        return self.cursor.lastrowid

    def add_image(self, name, camera_id, prior_q=None, prior_t=None):
        if prior_q is None:
            prior_q = [1.0, 0.0, 0.0, 0.0]
        if prior_t is None:
            prior_t = [0.0, 0.0, 0.0]

        self.cursor.execute(
            "INSERT INTO images VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (None, name, camera_id, *prior_q, *prior_t)
        )
        return self.cursor.lastrowid

    def add_keypoints(self, image_id, keypoints):
        if keypoints.dtype != np.float32:
            keypoints = keypoints.astype(np.float32)

        self.cursor.execute(
            "INSERT INTO keypoints VALUES (?, ?, ?, ?)",
            (image_id, keypoints.shape[0], keypoints.shape[1], keypoints.tobytes())
        )

    def add_matches(self, image_id1, image_id2, matches):
        pair_id = self.image_ids_to_pair_id(image_id1, image_id2)

        if matches.dtype != np.uint32:
            matches = matches.astype(np.uint32)

        self.cursor.execute(
            "INSERT OR REPLACE INTO matches VALUES (?, ?, ?, ?)",
            (pair_id, matches.shape[0], matches.shape[1], matches.tobytes())
        )

    @staticmethod
    def image_ids_to_pair_id(image_id1, image_id2):
        if image_id1 > image_id2:
            image_id1, image_id2 = image_id2, image_id1
        return image_id1 * 2147483648 + image_id2

    def commit(self):
        self.connection.commit()

    def close(self):
        self.connection.close()


# =========================================================
# H5 to Database Import
# =========================================================
CAMERA_MODEL_IDS = {
    'SIMPLE_PINHOLE': 0,
    'PINHOLE': 1,
    'SIMPLE_RADIAL': 2,
    'RADIAL': 3,
    'OPENCV': 4,
    'OPENCV_FISHEYE': 5,
}

def create_camera(db, image_path, camera_model):
    """Create camera entry"""
    img = Image.open(image_path)
    width, height = img.size

    # Simple radial model: f, cx, cy, k
    focal = max(width, height) * 1.2
    params = [focal, width/2, height/2, 0.0]

    model_id = CAMERA_MODEL_IDS.get(camera_model.upper(), 2)
    camera_id = db.add_camera(model_id, width, height, params)

    return camera_id

In [ ]:
def import_into_colmap(image_dir, feature_dir, database_path):
    """COLMAP Database Import - Multiple Cameras Support"""
    print("\n=== Creating COLMAP Database ===")

    if os.path.exists(database_path):
        os.remove(database_path)

    # Create empty database structure
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Create all tables
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS cameras (
            camera_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
            model INTEGER NOT NULL,
            width INTEGER NOT NULL,
            height INTEGER NOT NULL,
            params BLOB,
            prior_focal_length INTEGER NOT NULL
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS images (
            image_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
            name TEXT NOT NULL UNIQUE,
            camera_id INTEGER NOT NULL,
            prior_qw REAL,
            prior_qx REAL,
            prior_qy REAL,
            prior_qz REAL,
            prior_tx REAL,
            prior_ty REAL,
            prior_tz REAL,
            FOREIGN KEY(camera_id) REFERENCES cameras(camera_id)
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS keypoints (
            image_id INTEGER PRIMARY KEY NOT NULL,
            rows INTEGER NOT NULL,
            cols INTEGER NOT NULL,
            data BLOB,
            FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS matches (
            pair_id INTEGER PRIMARY KEY NOT NULL,
            rows INTEGER NOT NULL,
            cols INTEGER NOT NULL,
            data BLOB
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS two_view_geometries (
            pair_id INTEGER PRIMARY KEY NOT NULL,
            rows INTEGER NOT NULL,
            cols INTEGER NOT NULL,
            data BLOB,
            config INTEGER NOT NULL,
            F BLOB,
            E BLOB,
            H BLOB,
            qvec BLOB,
            tvec BLOB
        )
    """)

    # Load keypoints file
    kpts_file = os.path.join(feature_dir, 'keypoints.h5')
    matches_file = os.path.join(feature_dir, 'matches.h5')

    # Create cameras based on image sizes
    size_to_camera = {}  # (width, height) -> camera_id
    fname_to_id = {}
    image_id = 1

    with h5py.File(kpts_file, 'r') as f:
        print(f"Importing {len(f.keys())} images...")

        for filename in tqdm(f.keys(), desc="Adding images"):
            # Get image size
            image_path = os.path.join(image_dir, filename)
            try:
                img = Image.open(image_path)
                width, height = img.size
                img.close()
            except Exception as e:
                print(f"Warning: Cannot open {filename}: {e}")
                continue

            # Get or create camera for this size
            size_key = (width, height)
            if size_key not in size_to_camera:
                focal = max(width, height) * 1.2
                params = np.array([focal, width/2, height/2, 0.0], dtype=np.float64)

                cursor.execute(
                    "INSERT INTO cameras VALUES (?, ?, ?, ?, ?, ?)",
                    (None, 2, width, height, params.tobytes(), 1)  # 2 = SIMPLE_RADIAL
                )
                camera_id = cursor.lastrowid
                size_to_camera[size_key] = camera_id
            else:
                camera_id = size_to_camera[size_key]

            # Add image
            cursor.execute(
                "INSERT INTO images VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                (image_id, filename, camera_id, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
            )
            fname_to_id[filename] = image_id

            # Add keypoints
            kpts = f[filename][()].astype(np.float32)
            if len(kpts.shape) == 1:
                kpts = kpts.reshape(-1, 2)

            cursor.execute(
                "INSERT INTO keypoints VALUES (?, ?, ?, ?)",
                (image_id, kpts.shape[0], 2, kpts.tobytes())
            )

            image_id += 1

    print(f"\nCreated {len(size_to_camera)} camera(s) for different image sizes:")
    for size, cam_id in sorted(size_to_camera.items()):
        print(f"  Camera {cam_id}: {size[0]}x{size[1]}")

    # Add matches
    total_matches = 0
    total_match_count = 0
    with h5py.File(matches_file, 'r') as f:
        print(f"\nProcessing matches...")
        for key1 in tqdm(f.keys(), desc="Adding matches"):
            if key1 not in fname_to_id:
                continue
            for key2 in f[key1].keys():
                if key2 not in fname_to_id:
                    continue

                id1, id2 = fname_to_id[key1], fname_to_id[key2]
                if id1 >= id2:
                    continue

                matches = f[key1][key2][()].astype(np.uint32)
                if matches.shape[0] == 0:
                    continue

                pair_id = id1 * 2147483648 + id2
                cursor.execute(
                    "INSERT INTO matches VALUES (?, ?, ?, ?)",
                    (pair_id, matches.shape[0], 2, matches.tobytes())
                )
                total_matches += 1
                total_match_count += matches.shape[0]

    conn.commit()
    conn.close()

    print(f"\n✓ Database created: {database_path}")
    print(f"  Cameras: {len(size_to_camera)}")
    print(f"  Images: {len(fname_to_id)}")
    print(f"  Match pairs: {total_matches}")
    print(f"  Total matches: {total_match_count}")

    return fname_to_id

In [ ]:
def load_torch_image(fname, device=torch.device('cuda')):
    img = K.io.load_image(fname, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img


def extract_dino_embeddings(fnames, device=torch.device('cuda')):
    print("\n=== Stage 1: Extracting DINO Global Features ===")

    processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
    model = AutoModel.from_pretrained('facebook/dinov2-base')
    model = model.eval().to(device)

    global_descs = []
    for img_path in tqdm(fnames, desc="DINO extraction"):
        timg = load_torch_image(img_path, device)
        with torch.inference_mode():
            inputs = processor(images=timg, return_tensors="pt", do_rescale=False).to(device)
            outputs = model(**inputs)
            dino_feat = F.normalize(
                outputs.last_hidden_state[:,1:].max(dim=1)[0],
                dim=1, p=2
            )
        global_descs.append(dino_feat.detach().cpu())

    global_descs = torch.cat(global_descs, dim=0)
    print(f"Extracted global features: {global_descs.shape}")

    del model, processor
    torch.cuda.empty_cache()
    gc.collect()

    return global_descs


def build_topk_pairs(global_feats, device):
    print("\n=== Building Top-K Pairs from Global Features ===")

    g = global_feats.to(device)
    sim = g @ g.T
    sim.fill_diagonal_(-1)

    N = sim.size(0)
    k = min(CONFIG.GLOBAL_TOPK, N - 1)
    k = max(k, 1)

    topk_indices = torch.topk(sim, k, dim=1).indices.cpu()

    pairs = set()
    for i, neighbors in enumerate(topk_indices):
        for j in neighbors:
            j = j.item()
            if i < j:
                pairs.add((i, j))

    pairs = sorted(list(pairs))
    print(f"Initial pairs from global features: {len(pairs)}")
    return pairs


def extract_aliked_features(fnames, device=torch.device('cuda')):
    print("\n=== Stage 2: Extracting ALIKED Local Features ===")

    dtype = torch.float32
    extractor = ALIKED(
        model_name="aliked-n16",
        max_num_keypoints=CONFIG.max_num_keypoints,
        detection_threshold=0.01,
        resize=CONFIG.image_size
    ).eval().to(device, dtype)

    keypoints_dict = {}
    descriptors_dict = {}

    for img_path in tqdm(fnames, desc="ALIKED extraction"):
        key = os.path.basename(img_path)
        image = load_torch_image(img_path, device=device).to(dtype)

        with torch.inference_mode():
            feats = extractor.extract(image)
            kpts = feats['keypoints'].reshape(-1, 2).detach().cpu()
            descs = feats['descriptors'].reshape(-1, 128).detach().cpu()
            descs = F.normalize(descs, dim=1).half()

        keypoints_dict[key] = kpts.numpy()
        descriptors_dict[key] = descs

    print(f"Extracted features for {len(keypoints_dict)} images")

    del extractor
    torch.cuda.empty_cache()
    gc.collect()

    return keypoints_dict, descriptors_dict


def verify_pairs_with_local_features(pairs, fnames, descriptors_dict, device):
    print("\n=== Verifying Pairs with Local Features ===")

    verified_pairs = []

    for i, j in tqdm(pairs, desc="Local verification"):
        key1 = os.path.basename(fnames[i])
        key2 = os.path.basename(fnames[j])

        desc1 = descriptors_dict[key1].to(device)
        desc2 = descriptors_dict[key2].to(device)

        if desc1.size(0) == 0 or desc2.size(0) == 0:
            continue

        with torch.inference_mode():
            sim = desc1 @ desc2.T
            nn1 = torch.argmax(sim, dim=1)
            nn2 = torch.argmax(sim, dim=0)
            mutual = torch.arange(len(nn1), device=device) == nn2[nn1]
            n_matches = mutual.sum().item()

        if n_matches >= CONFIG.MATCH_THRESH:
            verified_pairs.append((i, j))

    print(f"Verified pairs: {len(verified_pairs)}")
    return verified_pairs

In [ ]:
def match_with_lightglue(verified_pairs, fnames, keypoints_dict, descriptors_dict,
                         output_dir, device=torch.device('cuda')):
    """Perform detailed matching using LightGlue - Fully Corrected Version"""
    print("\n=== Stage 3: Matching with LightGlue ===")

    os.makedirs(output_dir, exist_ok=True)

    lg_matcher = KF.LightGlueMatcher(
        "aliked", {
            "width_confidence": -1,
            "depth_confidence": -1,
            "mp": True if 'cuda' in str(device) else False
        }
    ).eval().to(device).half()

    print("Loaded LightGlue model")

    # Save keypoints
    kpts_h5_path = os.path.join(output_dir, 'keypoints.h5')
    with h5py.File(kpts_h5_path, 'w') as f:
        for img_path in fnames:
            key = os.path.basename(img_path)
            f.create_dataset(key, data=keypoints_dict[key])

    # Save matches
    matches_h5_path = os.path.join(output_dir, 'matches.h5')
    matched_pairs = 0
    skipped_pairs = 0
    total_matches = 0

    with h5py.File(matches_h5_path, 'w') as f_match:
        for i, j in tqdm(verified_pairs, desc="LightGlue matching"):
            key1 = os.path.basename(fnames[i])
            key2 = os.path.basename(fnames[j])

            kp1 = torch.from_numpy(keypoints_dict[key1]).to(device).half()
            kp2 = torch.from_numpy(keypoints_dict[key2]).to(device).half()
            desc1 = descriptors_dict[key1].to(device)
            desc2 = descriptors_dict[key2].to(device)

            if len(kp1) == 0 or len(kp2) == 0:
                skipped_pairs += 1
                continue

            with torch.inference_mode():
                try:
                    dists, idxs = lg_matcher(
                        desc1, desc2,
                        KF.laf_from_center_scale_ori(kp1[None]),
                        KF.laf_from_center_scale_ori(kp2[None])
                    )

                    # Check if matches were found
                    if idxs.numel() == 0:
                        skipped_pairs += 1
                        continue

                    # ★★★ Fix: Removed [0] ★★★
                    matches = idxs.cpu().numpy()  # (num_matches, 2)

                    # Check match count
                    num_matches = matches.shape[0]

                    if num_matches >= CONFIG.min_matches:
                        grp = f_match.require_group(key1)
                        grp.create_dataset(key2, data=matches)
                        matched_pairs += 1
                        total_matches += num_matches
                    else:
                        skipped_pairs += 1

                except Exception as e:
                    print(f"\nError matching {key1}-{key2}: {e}")
                    skipped_pairs += 1
                    continue

    del lg_matcher
    torch.cuda.empty_cache()
    gc.collect()

    print(f"\nMatching complete:")
    print(f"  Matched pairs: {matched_pairs}")
    print(f"  Skipped pairs: {skipped_pairs}")
    print(f"  Total matches: {total_matches}")
    print(f"  Average matches per pair: {total_matches/matched_pairs:.1f}" if matched_pairs > 0 else "")
    print(f"  Success rate: {matched_pairs/len(verified_pairs)*100:.1f}%")

    print(f"\nSaved keypoints to: {kpts_h5_path}")
    print(f"Saved matches to: {matches_h5_path}")




In [ ]:
def run_colmap_sequential(database_path, image_dir, output_dir):
    """Run COLMAP mapper with manual initial pair"""
    print("\n=== Stage 4: Running COLMAP Reconstruction ===")

    os.makedirs(output_dir, exist_ok=True)

    env = os.environ.copy()
    env['QT_QPA_PLATFORM'] = 'offscreen'

    from datetime import datetime, timezone
    print(f"🚀 Starting mapper at {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
    print("💡 Using manual initial pair: images 11-20 (5859 matches)")
    print()

    cmd_mapper = [
        'colmap', 'mapper',
        '--database_path', database_path,
        '--image_path', image_dir,
        '--output_path', output_dir,
        # 手動指定なし（自動）
        '--Mapper.ba_refine_focal_length', '1',
        '--Mapper.ba_refine_principal_point', '1',
        '--Mapper.ba_refine_extra_params', '1',
        # 標準的な設定
        '--Mapper.init_min_num_inliers', '50',
        '--Mapper.init_max_error', '8',
        '--Mapper.init_min_tri_angle', '4',
    ]

    import subprocess
    process = subprocess.Popen(
        cmd_mapper,
        env=env,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True
    )

    print("-" * 60)
    for line in iter(process.stdout.readline, ''):
        if line:
            print(line.rstrip(), flush=True)

    process.stdout.close()
    return_code = process.wait(timeout=3600)
    print("-" * 60)

    if return_code == 0:
        print(f"\n✅ COLMAP reconstruction saved to: {output_dir}")
        print(f"🕐 Completed at {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
    else:
        print(f"\n❌ COLMAP mapper failed with return code {return_code}")
        raise subprocess.CalledProcessError(return_code, cmd_mapper)

In [ ]:
def import_into_colmap(image_dir, feature_dir, database_path):
    """Import with camera grouping"""
    print("\n=== Creating COLMAP Database ===")

    if os.path.exists(database_path):
        os.remove(database_path)

    import cv2

    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Create tables
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS cameras (
            camera_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
            model INTEGER NOT NULL,
            width INTEGER NOT NULL,
            height INTEGER NOT NULL,
            params BLOB,
            prior_focal_length INTEGER NOT NULL
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS images (
            image_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
            name TEXT NOT NULL UNIQUE,
            camera_id INTEGER NOT NULL,
            prior_qw REAL,
            prior_qx REAL,
            prior_qy REAL,
            prior_qz REAL,
            prior_tx REAL,
            prior_ty REAL,
            prior_tz REAL,
            FOREIGN KEY(camera_id) REFERENCES cameras(camera_id)
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS keypoints (
            image_id INTEGER PRIMARY KEY NOT NULL,
            rows INTEGER NOT NULL,
            cols INTEGER NOT NULL,
            data BLOB,
            FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS matches (
            pair_id INTEGER PRIMARY KEY NOT NULL,
            rows INTEGER NOT NULL,
            cols INTEGER NOT NULL,
            data BLOB
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS two_view_geometries (
            pair_id INTEGER PRIMARY KEY NOT NULL,
            rows INTEGER NOT NULL,
            cols INTEGER NOT NULL,
            data BLOB,
            config INTEGER NOT NULL,
            F BLOB,
            E BLOB,
            H BLOB,
            qvec BLOB,
            tvec BLOB
        )
    """)

    kpts_file = os.path.join(feature_dir, 'keypoints.h5')
    matches_file = os.path.join(feature_dir, 'matches.h5')

    # Camera grouping function
    def get_camera_group(width, height, tolerance=100):
        """Group similar resolutions together"""
        w_group = round(width / tolerance) * tolerance
        h_group = round(height / tolerance) * tolerance
        return (w_group, h_group)

    # Add cameras and images
    size_to_camera = {}
    fname_to_id = {}
    image_id = 1

    with h5py.File(kpts_file, 'r') as f:
        print(f"Importing {len(f.keys())} images...")

        for filename in tqdm(f.keys(), desc="Adding images"):
            image_path = os.path.join(image_dir, filename)
            try:
                img = Image.open(image_path)
                width, height = img.size
                img.close()
            except:
                continue

            # Get grouped camera key
            size_key = get_camera_group(width, height, tolerance=50)

            if size_key not in size_to_camera:
                # Use group representative values
                focal = max(size_key[0], size_key[1])  # 1.2倍を削除
                params = np.array([focal, size_key[0]/2, size_key[1]/2, 0.0], dtype=np.float64)
                cursor.execute(
                    "INSERT INTO cameras VALUES (?, ?, ?, ?, ?, ?)",
                    (None, 2, size_key[0], size_key[1], params.tobytes(), 1)
                )
                size_to_camera[size_key] = cursor.lastrowid
                print(f"  Created camera group: {size_key[0]}x{size_key[1]}, focal={focal:.0f}")

            camera_id = size_to_camera[size_key]
            cursor.execute(
                "INSERT INTO images VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                (image_id, filename, camera_id, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
            )
            fname_to_id[filename] = image_id

            kpts = f[filename][()].astype(np.float32)
            if len(kpts.shape) == 1:
                kpts = kpts.reshape(-1, 2)
            cursor.execute(
                "INSERT INTO keypoints VALUES (?, ?, ?, ?)",
                (image_id, kpts.shape[0], 2, kpts.tobytes())
            )
            image_id += 1

    print(f"\n✅ Grouped into {len(size_to_camera)} camera(s) (from ~36 individual sizes)")

    # Geometric verification
    verified_count = 0

    with h5py.File(kpts_file, 'r') as f_kpts:
        with h5py.File(matches_file, 'r') as f_matches:
            print(f"\n🔧 Processing matches with geometric verification...")

            for key1 in tqdm(f_matches.keys(), desc="Verifying"):
                if key1 not in fname_to_id:
                    continue

                for key2 in f_matches[key1].keys():
                    if key2 not in fname_to_id:
                        continue

                    id1, id2 = fname_to_id[key1], fname_to_id[key2]
                    if id1 >= id2:
                        continue

                    matches = f_matches[key1][key2][()].astype(np.uint32)
                    if matches.shape[0] < 15:
                        continue

                    kpts1 = f_kpts[key1][()].astype(np.float64)
                    kpts2 = f_kpts[key2][()].astype(np.float64)

                    if len(kpts1.shape) == 1:
                        kpts1 = kpts1.reshape(-1, 2)
                    if len(kpts2.shape) == 1:
                        kpts2 = kpts2.reshape(-1, 2)

                    pts1 = kpts1[matches[:, 0]]
                    pts2 = kpts2[matches[:, 1]]

                    try:
                        F, mask = cv2.findFundamentalMat(
                            pts1, pts2,
                            cv2.FM_RANSAC,
                            3.0, 0.999
                        )

                        if F is None or mask is None:
                            continue

                        inliers = matches[mask.ravel() == 1]

                        if len(inliers) < 15:
                            continue

                        pair_id = id1 * 2147483648 + id2

                        cursor.execute(
                            "INSERT INTO matches VALUES (?, ?, ?, ?)",
                            (pair_id, len(inliers), 2, inliers.astype(np.uint32).tobytes())
                        )

                        cursor.execute(
                            "INSERT INTO two_view_geometries VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                            (pair_id, len(inliers), 2, inliers.astype(np.uint32).tobytes(),
                             2, F.astype(np.float64).tobytes(),
                             None, None, None, None)
                        )

                        verified_count += 1

                    except:
                        continue

    conn.commit()
    conn.close()

    print(f"\n✓ Database created: {database_path}")
    print(f"  Camera groups: {len(size_to_camera)}")
    print(f"  Images: {len(fname_to_id)}")
    print(f"  ✅ Geometrically verified pairs: {verified_count}")

    return fname_to_id

In [ ]:
def main_pipeline(image_dir, output_base_dir):
    """Complete pipeline"""

    from datetime import datetime, timezone

    print(f"\n🚀 Pipeline started at {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S UTC')}")

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Get images
    img_extensions = ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']
    fnames = []
    for ext in img_extensions:
        fnames.extend(glob.glob(os.path.join(image_dir, ext)))
    fnames = sorted(fnames)
    print(f"\n📸 Found {len(fnames)} images")

    if len(fnames) == 0:
        raise ValueError("No images found!")

    # Create directories
    feature_dir = os.path.join(output_base_dir, 'features')
    colmap_dir = os.path.join(output_base_dir, 'colmap')
    sparse_dir = os.path.join(colmap_dir, 'sparse')
    os.makedirs(feature_dir, exist_ok=True)
    os.makedirs(colmap_dir, exist_ok=True)

    # Stages 1-3: Feature extraction and matching
    print(f"\n⏰ Stage 1 started: {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
    global_feats = extract_dino_embeddings(fnames, device)

    print(f"\n⏰ Stage 2 started: {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
    initial_pairs = build_topk_pairs(global_feats, device)
    keypoints_dict, descriptors_dict = extract_aliked_features(fnames, device)

    print(f"\n⏰ Stage 3 started: {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
    verified_pairs = verify_pairs_with_local_features(
        initial_pairs, fnames, descriptors_dict, device
    )
    match_with_lightglue(
        verified_pairs, fnames, keypoints_dict, descriptors_dict,
        feature_dir, device
    )

    from datetime import datetime, timezone
    print()
    print(datetime.now(timezone.utc))

    print(f"\n⏰ Stage 4 started: {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
    # Stage 4: COLMAP Database + Reconstruction
    database_path = os.path.join(colmap_dir, 'database.db')
    #import_into_colmap(image_dir, feature_dir, database_path)
    import_into_colmap(image_dir, feature_dir, database_path)
    run_colmap_sequential(database_path, image_dir, sparse_dir)

    print("\n" + "="*60)
    print(f"✅ Pipeline Complete!")
    print(f"🕐 Finished at {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S UTC')}")
    print("="*60)


# Execute
IMAGE_DIR = "/content/drive/MyDrive/your_folder/fountain2"
OUTPUT_DIR = "/content/output"

main_pipeline(IMAGE_DIR, OUTPUT_DIR)

In [ ]:
def convert_cameras_to_pinhole(input_file, output_file):
    """Convert camera model to PINHOLE format, typically from OPENCV"""
    print(f"Reading camera file: {input_file}")

    with open(input_file, 'r') as f:
        lines = f.readlines()

    converted_count = 0
    with open(output_file, 'w') as f:
        for line in lines:
            # Write comments and empty lines directly
            if line.startswith('#') or line.strip() == '':
                f.write(line)
            else:
                parts = line.strip().split()
                if len(parts) >= 4:
                    cam_id = parts[0]
                    model = parts[1]
                    width = parts[2]
                    height = parts[3]
                    params = parts[4:]

                    # Convert to PINHOLE format
                    if model == "PINHOLE":
                        f.write(line)
                    elif model == "OPENCV":
                        # OPENCV: fx, fy, cx, cy, k1, k2, p1, p2 (only need first four for PINHOLE)
                        fx = params[0]
                        fy = params[1]
                        cx = params[2]
                        cy = params[3]
                        # PINHOLE: fx, fy, cx, cy
                        f.write(f"{cam_id} PINHOLE {width} {height} {fx} {fy} {cx} {cy}\n")
                        converted_count += 1
                    else:
                        # Convert other models by estimating PINHOLE parameters
                        # Set focal length to the max of width/height, and principal point to the center
                        fx = fy = max(float(width), float(height))
                        cx = float(width) / 2
                        cy = float(height) / 2
                        f.write(f"{cam_id} PINHOLE {width} {height} {fx} {fy} {cx} {cy}\n")
                        converted_count += 1
                else:
                    # Write lines that don't match the expected format
                    f.write(line)

    print(f"Converted {converted_count} cameras to PINHOLE format")



def prepare_gaussian_splatting_data(image_dir, colmap_model_dir):
    """Prepare data for Gaussian Splatting, structuring it in the expected format"""
    print("Preparing data for Gaussian Splatting...")

    # Assumes WORK_DIR is defined globally or passed
    data_dir = f"{WORK_DIR}/data/video"
    os.makedirs(f"{data_dir}/sparse/0", exist_ok=True)
    os.makedirs(f"{data_dir}/images", exist_ok=True)

    # Copy images
    print("Copying images...")
    img_count = 0
    for img_file in os.listdir(image_dir):
        if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            shutil.copy(
                os.path.join(image_dir, img_file),
                f"{data_dir}/images/{img_file}"
            )
            img_count += 1
    print(f"Copied {img_count} images")

    # Convert and copy camera file to PINHOLE format
    print("Converting camera model to PINHOLE format...")
    convert_cameras_to_pinhole(
        os.path.join(colmap_model_dir, 'cameras.txt'),
        f"{data_dir}/sparse/0/cameras.txt"
    )

    # Copy other files
    for filename in ['images.txt', 'points3D.txt']:
        src = os.path.join(colmap_model_dir, filename)
        dst = f"{data_dir}/sparse/0/{filename}"
        if os.path.exists(src):
            shutil.copy(src, dst)
            print(f"Copied {filename}")
        else:
            print(f"Warning: {filename} not found")

    print(f"Data preparation complete: {data_dir}")
    return data_dir



def train_gaussian_splatting(data_dir, iterations=3000):
    """Train the Gaussian Splatting model"""
    print(f"Training Gaussian Splatting model for {iterations} iterations...")

    # Assumes WORK_DIR is defined globally or passed
    model_path = f"{WORK_DIR}/output/video"

    cmd = [
        sys.executable, 'train.py',
        '-s', data_dir,
        '-m', model_path,
        '--iterations', str(iterations),
        '--eval' # Optionally run an evaluation phase
    ]

    # Execute the training script from the WORK_DIR
    subprocess.run(cmd, cwd=WORK_DIR, check=True)

    return model_path



def render_video(model_path, output_video_path, iteration=3000):
    """Generate video from the trained model by rendering a sequence of views"""
    print("Rendering video...")

    # Execute rendering
    cmd = [
        sys.executable, 'render.py',
        '-m', model_path,
        '--iteration', str(iteration)
    ]

    # Execute the rendering script from the WORK_DIR
    subprocess.run(cmd, cwd=WORK_DIR, check=True)

    # Find the rendering directory
    possible_dirs = [
        f"{model_path}/test/ours_{iteration}/renders",
        f"{model_path}/train/ours_{iteration}/renders",
    ]

    render_dir = None
    for test_dir in possible_dirs:
        if os.path.exists(test_dir):
            render_dir = test_dir
            print(f"Rendering directory found: {render_dir}")
            break

    if render_dir and os.path.exists(render_dir):
        # Sort rendered PNG images for correct video sequence
        render_imgs = sorted([f for f in os.listdir(render_dir) if f.endswith('.png')])

        if render_imgs:
            print(f"Found {len(render_imgs)} rendered images")

            # Create video with ffmpeg
            # -y: overwrite output file without asking
            # -framerate 30: set input framerate to 30 FPS
            # -pattern_type glob -i: use glob pattern to specify input images
            # -c:v libx264: use h.264 video codec
            # -pix_fmt yuv420p: use a pixel format compatible with most players
            # -crf 18: Constant Rate Factor (lower is higher quality, 18 is generally high quality)
            subprocess.run([
                'ffmpeg', '-y',
                '-framerate', '30',
                '-pattern_type', 'glob',
                '-i', f"{render_dir}/*.png",
                '-c:v', 'libx264',
                '-pix_fmt', 'yuv420p',
                '-crf', '18',
                output_video_path
            ], check=True)

            print(f"Video saved: {output_video_path}")
            return True

    print("Error: Rendering directory not found or no images rendered")
    return False



def create_gif(video_path, gif_path):
    """Create an animated GIF from an MP4 video file"""
    print("Creating animated GIF...")

    # ffmpeg command to create a GIF
    # -vf: video filter graph
    # setpts=8*PTS: slows down the video by a factor of 8 (8x original duration)
    # fps=10: set output frame rate to 10 FPS
    # scale=720:-1:flags=lanczos: resize to 720px width, auto height, using Lanczos resampling
    # -loop 0: loop the GIF indefinitely
    subprocess.run([
        'ffmpeg', '-y',
        '-i', video_path,
        '-vf', 'setpts=8*PTS,fps=10,scale=720:-1:flags=lanczos',
        '-loop', '0',
        gif_path
    ], check=True)

    if os.path.exists(gif_path):
        size_mb = os.path.getsize(gif_path) / (1024 * 1024)
        print(f"GIF creation complete: {gif_path} ({size_mb:.2f} MB)")
        return True

    return False


In [ ]:
model_path = train_gaussian_splatting(data_dir, iterations=1000)

# Step 6: Render Video
os.makedirs(OUTPUT_DIR, exist_ok=True)
output_video = f"{OUTPUT_DIR}/gaussian_splatting_video.mp4"
success = render_video(model_path, output_video, iteration=1000)


In [ ]:
def diagnose_specific_pair(database_path, id1, id2):
    """Diagnose a specific image pair"""
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    print(f"\n🔍 Diagnosing pair {id1}-{id2}")
    print("="*60)

    # Get match info
    pair_id = id1 * 2147483648 + id2
    cursor.execute(
        "SELECT rows, F FROM two_view_geometries WHERE pair_id=?",
        (pair_id,)
    )
    result = cursor.fetchone()

    if result:
        rows, F_blob = result
        print(f"  Matches: {rows}")

        if F_blob:
            F = np.frombuffer(F_blob, dtype=np.float64).reshape(3, 3)
            print(f"  F matrix exists: {F.shape}")
            print(f"  F matrix:\n{F}")
        else:
            print("  ⚠️ F matrix is NULL!")
    else:
        print(f"  ⚠️ Pair not found in two_view_geometries!")

    # Get camera info
    cursor.execute("SELECT c.* FROM cameras c JOIN images i ON c.camera_id = i.camera_id WHERE i.image_id IN (?, ?)", (id1, id2))
    print("\n  Cameras:")
    for row in cursor.fetchall():
        cam_id, model, w, h, params_blob, prior = row
        params = np.frombuffer(params_blob, dtype=np.float64)
        print(f"    Camera {cam_id}: {w}x{h}, model={model}, params={params}")

    conn.close()
    print("="*60)

# 実行
diagnose_specific_pair('/content/output/colmap/database.db', 11, 20)

In [ ]:
import sqlite3
import numpy as np

def diagnose_database(database_path):
    """Diagnose why COLMAP can't find initial pair"""
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    print("\n🔍 Database Diagnosis")
    print("="*60)

    # Get match statistics
    cursor.execute("""
        SELECT pair_id, rows, config
        FROM two_view_geometries
        ORDER BY rows DESC
        LIMIT 10
    """)

    print("\nTop 10 matches by count:")
    for pair_id, rows, config in cursor.fetchall():
        image_id2 = pair_id % 2147483648
        image_id1 = (pair_id - image_id2) // 2147483648
        print(f"  Images {image_id1}-{image_id2}: {rows} matches, config={config}")

    # Get image and camera info
    cursor.execute("""
        SELECT i.image_id, i.name, i.camera_id, c.width, c.height
        FROM images i
        JOIN cameras c ON i.camera_id = c.camera_id
        LIMIT 5
    """)

    print("\nSample images:")
    for img_id, name, cam_id, w, h in cursor.fetchall():
        print(f"  Image {img_id}: {name}, camera {cam_id} ({w}x{h})")

    conn.close()
    print("="*60)

# 実行
diagnose_database('/content/output/colmap/database.db')